In [1]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


# About Dataset
## Spotify Recommandation


The dataset used in this analysis was collected from the Spotify API, which provides access to a vast repository of music metadata. The API's data was utilized to gather information on various song attributes, including danceability, energy, key, and more.

## 1. Data Collection
1.1 Playlist creation
I collected 100 liked songs and 95 disliked songs

For those I like , I made a playlist of my favorite 100 songs. It is mainly French Rap , sometimes American rap , rock or electro music.

For those I dislike , I collected songs from various kind of music so the model will have a broader view of what I don't like

There is :

25 metal songs ( Cannibal Corps )
20 " I don't like " rap songs ( PNL )
25 classical songs
25 Disco songs
I didn't include any Pop song because I'm kinda neutral about it

### 1.2 Getting the ID's
From the Spotify's API "Get a playlist's Items" , I turned the playlists into json formatted data which cointains the ID and the name of each track ( ids/yes.py and ids/no.py ). NB : on the website , specify "items(track(id,name))" in the fields format , to avoid being overwhelmed by useless data.

With a script ( ids/ids_to_data.py ) , I turned the json data into a long string with each ID separated with a comma.

### 1.3 Getting the statistics
Now I just had to enter the strings into the Spotify API "Get Audio Features from several tracks" and get my data files ( data/good.json and data/dislike.json )

## 2. Data features
From Spotify's API documentation :

**acousticness :** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

**danceability :** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

**duration_ms :** The duration of the track in milliseconds.

**energy :** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

**instrumentalness :** Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

**key :** The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on.
liveness : Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.

**loudness :** The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.

**mode :** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

**speechiness :** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

**tempo :** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

**time_signature :** An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).

**valence :** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
And the variable that has to be predicted :

**liked :** 1 for liked songs , 0 for disliked songs



The dataset is available on [Kaggle website](https://www.kaggle.com/datasets/bricevergnou/spotify-recommendation/data)

In [2]:
import pandas as pd


df = pd.read_csv("data.csv")
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,liked
0,0.803,0.6240,7,-6.764,0,0.0477,0.451,0.000734,0.1000,0.6280,95.968,304524,4,0
1,0.762,0.7030,10,-7.951,0,0.3060,0.206,0.000000,0.0912,0.5190,151.329,247178,4,1
2,0.261,0.0149,1,-27.528,1,0.0419,0.992,0.897000,0.1020,0.0382,75.296,286987,4,0
3,0.722,0.7360,3,-6.994,0,0.0585,0.431,0.000001,0.1230,0.5820,89.860,208920,4,1
4,0.787,0.5720,1,-7.516,1,0.2220,0.145,0.000000,0.0753,0.6470,155.117,179413,4,1


# 1. XGBOOST

In [21]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score


# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('liked', axis=1), df['liked'], test_size=0.2, random_state=42)

# create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# set hyperparameters for XGBoost
params = {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100, 'objective': 'binary:logistic', 'eval_metric': 'auc'}

# train XGBoost model
bst = xgb.train(params, dtrain, num_boost_round=100)

# make predictions on test set
y_pred = bst.predict(dtest)

# evaluate model using AUC-ROC
auc = roc_auc_score(y_test, y_pred)
print(f'AUC-ROC: {auc:.3f}')
new_song_features = pd.DataFrame({'danceability':[0.803], 'energy':[0.624], 'key':[7], 'loudness':[6.764],
                                  'mode':[0],'speechiness':[0.0477],'acousticness':[0.451],'instrumentalness':[0.000734],
                                  'liveness':[0.01],'valence':[0.628],'tempo':[95.968],'duration_ms':[304524],'time_signature':[4]})
new_song_dm = xgb.DMatrix(new_song_features)
new_song_pred = bst.predict(new_song_dm)
print(f'Likelihood of liking: {new_song_pred[0]:.3f}')

AUC-ROC: 0.974
Likelihood of liking: 0.082


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [06:38:36] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


# 2. Nerual Network

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import xgboost as xgb
from keras.callbacks import EarlyStopping

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('liked', axis=1), df['liked'], test_size=0.2, random_state=42)

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
new_song_scaled = scaler.transform(new_song_features)

# Convert the scaled arrays to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
new_song_scaled = np.array(new_song_scaled)

# Create a neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=128, validation_data=(X_test_scaled, y_test), callbacks=[EarlyStopping(patience=3)])

# Make a prediction on the new song
new_song_pred = model.predict(new_song_scaled)[0].item()
print(f'Likelihood of liking: {new_song_pred:.3f}')

# Make a prediction on the new song
new_song_features = np.array([[0.803, 0.6240, 7, -6.764, 0, 0.0477, 0.451, 0.000734, 0.1000, 0.6280, 95.968, 304524, 4]])
new_song_scaled = scaler.transform(new_song_features)
new_song_pred = model.predict(new_song_scaled)[0].item()
print(f'Likelihood of liking: {new_song_pred:.3f}')

Epoch 1/10
2/2 [==============================] - 2s 756ms/step - loss: 3.2532 - accuracy: 0.3846 - val_loss: 2.4691 - val_accuracy: 0.3846
Epoch 2/10
2/2 [==============================] - 0s 224ms/step - loss: 2.1325 - accuracy: 0.4167 - val_loss: 2.0313 - val_accuracy: 0.3846
Epoch 3/10
2/2 [==============================] - 0s 175ms/step - loss: 1.4847 - accuracy: 0.4167 - val_loss: 1.2693 - val_accuracy: 0.4872
Epoch 4/10
2/2 [==============================] - 0s 104ms/step - loss: 1.2730 - accuracy: 0.4423 - val_loss: 1.1521 - val_accuracy: 0.5385
Epoch 5/10
2/2 [==============================] - 0s 106ms/step - loss: 1.1069 - accuracy: 0.4744 - val_loss: 0.7524 - val_accuracy: 0.5385
Epoch 6/10
2/2 [==============================] - 0s 275ms/step - loss: 1.0332 - accuracy: 0.5256 - val_loss: 0.6770 - val_accuracy: 0.5385
Epoch 7/10
2/2 [==============================] - 0s 279ms/step - loss: 0.8178 - accuracy: 0.5897 - val_loss: 0.6179 - val_accuracy: 0.5897
Epoch 8/10
2/2 [====

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 100ms/step
Likelihood of liking: 0.290


# 3. Ensemble Models

In [24]:
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier , BaggingClassifier , AdaBoostClassifier
from sklearn.metrics import accuracy_score , roc_auc_score


rf_clf = RandomForestClassifier(n_estimators=100)
extratree_clf = ExtraTreesClassifier(n_estimators=100)
bg_clf = BaggingClassifier(n_estimators=100)
ada_clf = AdaBoostClassifier(n_estimators=100)
rf_clf.fit(X_train , y_train)
extratree_clf.fit(X_train , y_train)
bg_clf.fit(X_train , y_train)
ada_clf.fit(X_train , y_train)
y_preds_rf = rf_clf.predict(X_test)
y_preds_et = extratree_clf.predict(X_test)
y_preds_bg = bg_clf.predict(X_test)
y_preds_ada = ada_clf.predict(X_test)
print("Accuracy Score of the Random Forest Model", accuracy_score(y_test, y_preds_rf))
print("Accuracy Score of the Extre Tree Classifier Model", accuracy_score(y_test, y_preds_et))
print("Accuracy Score of the Bagging Classifier Model", accuracy_score(y_test, y_preds_bg))
print("Accuracy Score of the AdaBoost Classifier Model", accuracy_score(y_test, y_preds_ada))
print("ROC AUC Score of the Random Forest Model", roc_auc_score(y_test, y_preds_rf))
print("ROC AUC Score of the Extre Tree Classifier Model", roc_auc_score(y_test, y_preds_et))
print("ROC AUC Score of the Bagging Classifier Model", roc_auc_score(y_test, y_preds_bg))
print("ROC AUC Score of the AdaBoost Classifier Model", roc_auc_score(y_test, y_preds_ada))


Accuracy Score of the Random Forest Model 0.9230769230769231
Accuracy Score of the Extre Tree Classifier Model 0.9487179487179487
Accuracy Score of the Bagging Classifier Model 0.8717948717948718
Accuracy Score of the AdaBoost Classifier Model 0.8717948717948718
ROC AUC Score of the Random Forest Model 0.9236842105263158
ROC AUC Score of the Extre Tree Classifier Model 0.9486842105263158
ROC AUC Score of the Bagging Classifier Model 0.8723684210526317
ROC AUC Score of the AdaBoost Classifier Model 0.8723684210526317


# 4. LGBM Classifier

In [25]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train , y_train)
y_preds_lgbm = lgbm_clf.predict(X_test)
print("Accuracy Score of the LGBM Model", accuracy_score(y_test, y_preds_lgbm))
print("ROC AUC Score of the LGBM Model", roc_auc_score(y_test, y_preds_lgbm))

[LightGBM] [Info] Number of positive: 80, number of negative: 76
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 514
[LightGBM] [Info] Number of data points in the train set: 156, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.512821 -> initscore=0.051293
[LightGBM] [Info] Start training from score 0.051293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

# Conclusion (XGBoost and Nural Network)
By comparing the performance of the XGBoost and Neural Network models on the given dataset, we can conclude that the Neural Network model performed better in predicting whether a user would like a song or not. When fed known data, the Neural Network model accurately predicted a low likelihood of the user liking the song, whereas the XGBoost model incorrectly predicted a higher likelihood. This discrepancy highlights the importance of model selection and evaluation in machine learning applications.

Models: Neural Network Model and XGBoost